<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=897265dc7c2458faba920683c2a787c9c14ee771d3d13ce54ea123857cf7c84d
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-03 11:00:50
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.40 C
-------------------
Today PnL: 19.78 K (0.14%)
Current PnL: -21.13 L (-14.03%)
CY Booked + Current PnL: -9.69 L (-6.43%)
-------------------
Total profit:  1.29 L
Total loss:  -22.43 L
-------------------
Total Booked + Current PnL: 17.29 L (13.98%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.15%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.45 L (62.97%)
Deployed:  1.24 C
Current:  1.40 C
CAGR/XIRR %: 7.75%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-41.18,70.0,H-MC,13.03,117798.0,17294.0,11815.0,3.62,17.21,10.03,28.96,79.0,1.46,0.84,57.02,MH,ATH,METALS
77,TTKPRESTIG,770.00,94.45,40.0,M-SC,1.70,83788.0,-16989.0,17084.0,0.40,-16.86,20.39,0.09,245.0,-0.99,0.60,10.08,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,8.25,43.0,H-MC,2.90,186810.0,22140.0,21184.0,-0.52,13.45,11.34,26.31,89.0,1.05,1.34,22.50,X40N,NTT,AC
42,ITC,452.00,-38.77,46.0,H-LC,2.17,197727.0,-2411.0,23292.0,-0.31,-1.20,11.78,10.44,4.0,-0.10,1.41,4.29,X40,NTT,FMCG
34,ICICIGI,2260.25,-17.79,55.0,H-MC,1.75,141273.0,5140.0,25980.0,0.29,3.78,18.39,22.86,91.0,0.20,1.01,16.87,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-188.64,81.0,M-SC,8.40,152622.0,3402.0,140779.0,5.42,2.28,92.24,96.62,208.0,0.02,1.09,65.17,XY25,NTT,FINANCE
71,TANLA,1963.11,-17.86,58.0,H-SC,18.54,219861.0,-50900.0,380843.0,4.07,-18.80,173.22,121.86,133.0,-0.13,1.57,71.99,AR,ATH,IT
50,NATIONALUM,244.55,-41.18,70.0,H-MC,13.03,117798.0,17294.0,11815.0,3.62,17.21,10.03,28.96,79.0,1.46,0.84,57.02,MH,ATH,METALS
24,DEN,75.00,99.48,48.0,M-SC,1.94,55952.0,-28512.0,64048.0,2.85,-33.76,114.47,42.07,237.0,-0.45,0.40,18.82,AR,NTT,ENTERTAINMENT
12,BATAINDIA,2096.00,-32.72,53.0,M-SC,4.77,100513.0,-28157.0,73455.0,1.93,-21.88,73.08,35.20,219.0,-0.38,0.72,15.37,X40,NTT,FOOTWEAR


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1215.79,64.0,M-SC,18.13,178967.0,-2176.0,47641.0,-2.23,-1.20,26.62,25.10,235.0,-0.05,1.28,43.42,XY24,NTT,HEALTHCARE
26,GREENPANEL,537.00,219.12,53.0,M-SC,2.13,142520.0,-40558.0,119532.0,-1.77,-22.15,83.87,43.14,230.0,-0.34,1.02,34.06,XY24,NTT,MISC
49,MASFIN,398.61,-20.51,40.0,H-SC,11.54,90510.0,-7470.0,29072.0,-1.68,-7.62,32.12,22.05,152.0,-0.26,0.65,31.56,XR,ATH,FINANCE
66,SIS,528.00,2070.98,40.0,H-SC,3.70,87605.0,-23427.0,47035.0,-1.59,-21.10,53.69,21.26,156.0,-0.50,0.63,18.36,OX40N,NTT,MISC
20,COALINDIA,497.97,5.97,41.0,L-LC,4.23,141728.0,-8732.0,42518.0,-1.40,-5.80,30.00,22.46,167.0,-0.21,1.01,11.23,XY25,ATH,MINING


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.0,-21.34,47.0,H-MC,2.17,135430.0,-358.0,43107.0,0.44,-0.26,31.83,31.48,107.0,-0.01,0.97,11.82,X40,ATH,INSURANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,105.37,64.0,H-SC,12.08,129816.0,-12663.0,32181.0,-0.31,-8.89,24.79,13.70,163.0,-0.39,0.93,53.22,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.24,59.0,H-SC,1.84,226143.0,-43524.0,79851.0,0.82,-16.14,35.31,13.47,138.0,-0.55,1.62,14.38,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,94.45,40.0,M-SC,1.70,83788.0,-16989.0,17084.0,0.40,-16.86,20.39,0.09,245.0,-0.99,0.60,10.08,OX40N,NTT,DURABLES
18,CERA,9475.0,-22.43,40.0,H-SC,1.47,141576.0,-34327.0,76352.0,0.81,-19.51,53.93,23.89,149.0,-0.45,1.01,22.27,OX40N,NTT,CERAMICS
66,SIS,528.0,2070.98,40.0,H-SC,3.70,87605.0,-23427.0,47035.0,-1.59,-21.10,53.69,21.26,156.0,-0.50,0.63,18.36,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,49.96,64.0,M-MC,11.99,236568.0,11606.0,156040.0,1.82,5.16,65.96,74.53,192.0,0.07,1.69,39.08,XY24,BTT,STEEL
32,HINDZINC,730.22,28.52,64.0,M-LC,3.86,209387.0,4311.0,108253.0,-0.23,2.10,51.70,54.89,52.0,0.04,1.50,26.70,X5K,ATH,METALS
14,BLUESTARCO,2080.00,8.25,43.0,H-MC,2.90,186810.0,22140.0,21184.0,-0.52,13.45,11.34,26.31,89.0,1.05,1.34,22.50,X40N,NTT,AC
84,VOLTAS,1530.00,-3.46,43.0,H-MC,2.19,202500.0,10758.0,26993.0,-0.26,5.61,13.33,19.69,99.0,0.40,1.45,12.79,XY25,NTT,AC
17,CAMS,5211.76,-5.83,46.0,H-SC,1.97,107097.0,5093.0,38833.0,0.04,4.99,36.26,43.06,122.0,0.13,0.77,22.96,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-41.18,70.0,H-MC,13.03,117798.0,17294.0,11815.0,3.62,17.21,10.03,28.96,79.0,1.46,0.84,57.02,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,65.0,H-MC,10.00,190743.0,10935.0,101475.0,-0.49,6.08,53.20,62.52,88.0,0.11,1.36,42.24,XR,NTT,BANKS
32,HINDZINC,730.22,28.52,64.0,M-LC,3.86,209387.0,4311.0,108253.0,-0.23,2.10,51.70,54.89,52.0,0.04,1.50,26.70,X5K,ATH,METALS
37,INDIAMART,4850.92,-54.64,36.0,H-SC,9.40,126723.0,3387.0,130373.0,-0.33,2.75,102.88,108.45,119.0,0.03,0.91,25.71,AR,ATH,MISC
86,WIPRO,420.00,-15.61,40.0,M-LC,5.87,148695.0,-2250.0,111700.0,-0.51,-1.49,75.12,72.51,53.0,-0.02,1.06,4.44,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.42,29.0,H-LC,5.80,206008.0,-45760.0,98966.0,0.22,-18.18,48.04,21.13,27.0,-0.46,1.47,10.81,X40,ATH,PAINTS
83,VBL,671.64,-22.30,30.0,H-LC,9.59,279667.0,-36175.0,148839.0,-1.04,-11.45,53.22,35.67,5.0,-0.24,2.00,0.81,X40N,ATH,FMCG
74,TCS,4476.75,-31.58,31.0,H-LC,12.47,272290.0,-73666.0,148534.0,-0.60,-21.29,54.55,21.64,1.0,-0.50,1.95,0.29,X40,ATH,IT
30,HCLTECH,1943.91,-4.95,33.0,H-LC,10.31,217414.0,-24510.0,87770.0,-0.34,-10.13,40.37,26.15,8.0,-0.28,1.55,2.74,X40,ATH,IT
40,INFY,2275.00,-21.03,38.0,H-LC,8.24,307082.0,-5806.0,177493.0,-0.28,-1.86,57.80,54.87,3.0,-0.03,2.20,4.67,X40,BTT,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,-2.83,44.0,H-LC,1.74,156320.0,-29775.0,77253.0,0.16,-16.00,49.42,25.51,15.0,-0.39,1.12,15.20,AR,ATH,ELECTRICAL
48,LTIM,7230.2,-6.94,44.0,H-LC,1.81,231052.0,-15099.0,94315.0,0.22,-6.13,40.82,32.18,16.0,-0.16,1.65,27.70,X200,ATH,IT
42,ITC,452.0,-38.77,46.0,H-LC,2.17,197727.0,-2411.0,23292.0,-0.31,-1.20,11.78,10.44,4.0,-0.10,1.41,4.29,X40,NTT,FMCG
51,NESTLEIND,1377.0,-11.91,43.0,H-LC,3.40,271183.0,5757.0,51037.0,-0.64,2.17,18.82,21.40,11.0,0.11,1.94,8.96,XY25,NTT,FMCG
57,RELIANCE,1533.0,-15.17,38.0,H-LC,3.54,212098.0,1732.0,27042.0,-0.66,0.82,12.75,13.68,37.0,0.06,1.52,17.48,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,31.0,H-LC,12.47,272290.0,-73666.0,148534.0,-0.60,-21.29,54.55,21.64,1.0,-0.50,1.95,0.29,X40,ATH,IT
83,VBL,671.64,-22.30,30.0,H-LC,9.59,279667.0,-36175.0,148839.0,-1.04,-11.45,53.22,35.67,5.0,-0.24,2.00,0.81,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.95,33.0,H-LC,10.31,217414.0,-24510.0,87770.0,-0.34,-10.13,40.37,26.15,8.0,-0.28,1.55,2.74,X40,ATH,IT
42,ITC,452.00,-38.77,46.0,H-LC,2.17,197727.0,-2411.0,23292.0,-0.31,-1.20,11.78,10.44,4.0,-0.10,1.41,4.29,X40,NTT,FMCG
40,INFY,2275.00,-21.03,38.0,H-LC,8.24,307082.0,-5806.0,177493.0,-0.28,-1.86,57.80,54.87,3.0,-0.03,2.20,4.67,X40,BTT,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.83,44.0,H-LC,1.74,156320.0,-29775.0,77253.0,0.16,-16.00,49.42,25.51,15.0,-0.39,1.12,15.20,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,46.0,H-LC,2.17,197727.0,-2411.0,23292.0,-0.31,-1.20,11.78,10.44,4.0,-0.10,1.41,4.29,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.42,29.0,H-LC,5.80,206008.0,-45760.0,98966.0,0.22,-18.18,48.04,21.13,27.0,-0.46,1.47,10.81,X40,ATH,PAINTS
19,CIPLA,1795.00,-22.18,43.0,H-LC,3.90,207199.0,2699.0,38725.0,-0.04,1.32,18.69,20.25,10.0,0.07,1.48,10.17,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-15.17,38.0,H-LC,3.54,212098.0,1732.0,27042.0,-0.66,0.82,12.75,13.68,37.0,0.06,1.52,17.48,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,31.0,H-LC,12.47,272290.0,-73666.0,148534.0,-0.60,-21.29,54.55,21.64,1.0,-0.50,1.95,0.29,X40,ATH,IT
40,INFY,2275.00,-21.03,38.0,H-LC,8.24,307082.0,-5806.0,177493.0,-0.28,-1.86,57.80,54.87,3.0,-0.03,2.20,4.67,X40,BTT,IT
42,ITC,452.00,-38.77,46.0,H-LC,2.17,197727.0,-2411.0,23292.0,-0.31,-1.20,11.78,10.44,4.0,-0.10,1.41,4.29,X40,NTT,FMCG
83,VBL,671.64,-22.30,30.0,H-LC,9.59,279667.0,-36175.0,148839.0,-1.04,-11.45,53.22,35.67,5.0,-0.24,2.00,0.81,X40N,ATH,FMCG
1,ABB,7934.00,-40.55,44.0,H-LC,7.66,248280.0,-13339.0,132557.0,-0.51,-5.10,53.39,45.57,7.0,-0.10,1.77,5.65,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,47.0,L-SC,37.56,80640.0,-32909.0,72963.0,1.40,-28.98,90.48,35.27,268.0,-0.45,0.58,97.37,XR,NTT,HOTELS
49,MASFIN,398.61,-20.51,40.0,H-SC,11.54,90510.0,-7470.0,29072.0,-1.68,-7.62,32.12,22.05,152.0,-0.26,0.65,31.56,XR,ATH,FINANCE
50,NATIONALUM,244.55,-41.18,70.0,H-MC,13.03,117798.0,17294.0,11815.0,3.62,17.21,10.03,28.96,79.0,1.46,0.84,57.02,MH,ATH,METALS
78,UJJIVANSFB,60.00,105.37,64.0,H-SC,12.08,129816.0,-12663.0,32181.0,-0.31,-8.89,24.79,13.70,163.0,-0.39,0.93,53.22,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-188.64,81.0,M-SC,8.40,152622.0,3402.0,140779.0,5.42,2.28,92.24,96.62,208.0,0.02,1.09,65.17,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1215.79,64.0,M-SC,18.13,178967.0,-2176.0,47641.0,-2.23,-1.20,26.62,25.10,235.0,-0.05,1.28,43.42,XY24,NTT,HEALTHCARE
69,SURYODAY,240.00,75.44,81.0,H-SC,3.93,160609.0,-18462.0,84191.0,1.20,-10.31,52.42,36.71,135.0,-0.22,1.15,59.36,XR,NTT,BANKS
61,SAMMAANCAP,326.00,-188.64,81.0,M-SC,8.40,152622.0,3402.0,140779.0,5.42,2.28,92.24,96.62,208.0,0.02,1.09,65.17,XY25,NTT,FINANCE
71,TANLA,1963.11,-17.86,58.0,H-SC,18.54,219861.0,-50900.0,380843.0,4.07,-18.80,173.22,121.86,133.0,-0.13,1.57,71.99,AR,ATH,IT
7,ATULAUTO,844.00,3586.49,50.0,M-SC,7.85,165461.0,-31671.0,121498.0,-0.72,-16.07,73.43,45.57,236.0,-0.26,1.18,17.31,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.97
1,25,43.12
2,50,74.79


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.35
LC    31.95
MC    23.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.91
X40      14.44
X40N     11.23
XR       11.12
XY25     10.41
AR        8.17
OX40N     7.79
MH        1.65
X200      1.65
X5K       1.50
SR        1.16
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.15
H-LC    25.38
H-MC    20.43
M-SC    13.75
M-LC     5.56
M-MC     3.00
L-SC     1.45
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.77,-6.59,41.78
IT,12.51,-20.09,85.57
FINANCE,10.43,-15.57,68.06
MISC,6.94,-18.74,84.76
BANKS,6.44,-10.83,69.13
PAINTS,5.58,-18.16,35.77
ELECTRICAL,5.33,-10.55,49.33
INSURANCE,3.75,-5.36,44.74
AUTO,3.57,-14.97,69.93


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3248732.0,22
XR,1355423.0,14
AR,1189602.0,9
X40,830029.0,10
X40N,635866.0,9
OX40N,558114.0,10
XY25,481944.0,7
SR,259236.0,2
X5K,108253.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3720611.0,29
M-SC,1507834.0,17
H-LC,1269906.0,15
H-MC,1248185.0,15
M-LC,372665.0,4
M-MC,362887.0,2
L-SC,259820.0,3
L-MC,60139.0,1
L-LC,42518.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1230067.0      6
M-SC       XY24       856700.0      7
H-SC       AR         812474.0      5
           XR         780679.0      7
H-MC       XY24       572069.0      4
H-LC       X40        536055.0      5
M-MC       XY24       362887.0      2
H-SC       X40N       311179.0      4
           SR         259236.0      2
           OX40N      255740.0      4
H-LC       X40N       238005.0      3
H-MC       X40        220519.0      4
H-LC       AR         209810.0      2
H-MC       XY25       181223.0      2
L-SC       XR         170853.0      2
M-SC       AR         167318.0      2
M-LC       XY24       152712.0      2
M-SC       XY25       140779.0      1
           OX40N      139005.0      4
           XR         130577.0      2
H-LC       XY25       117424.0      3
M-LC       XR         111700.0      1
           X5K        108253.0      1
H-MC       XR         101475.0      1
H-LC       X200        94315.0      1
L-SC       OX40N       88967.0      1
H-MC       X40N        86682.0      2
           OX40N       74402.0      1
H-LC       XY24        74297.0      1
M-SC       X40         73455.0      1
H-SC       MH          71236.0      1
L-MC       XR          60139.0      1
L-LC       XY25        42518.0      1
H-MC       MH          11815.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
